In [1]:
### Atom class + readframe + lines_in_file functions - used to process dump files

import numpy as np
import operator # do we need this?

class Atom:
    """ A Class for storing atom information """

    def __init__(self):
        """ Initialise the class """
        self.id = 0                                              # id of the atom
        self.type = 0                                            # type of the atom
        self.L = np.array([0.0,0.0,0.0],dtype=np.float64)        # size of simulation box
        self.half_L = self.L / 2                                 # half the size of simulation box
        self.x = np.array([0.0,0.0,0.0],dtype=np.float64)        # position of the atom
        self.image = np.array([0,0,0],dtype=np.int32)            # image flags for atoms
        self.x_unwrap = np.array([0.0,0.0,0.0],dtype=np.float64) # position of the atom - unwrapped coords
        self.unwrap_flag = False

    
    def sep(self, atom2):
        """ Takes in self and atom2 and finds separation between them taking into account periodic BCs """

        dx = self.x[0] - atom2.x[0]
        dy = self.x[1] - atom2.x[1]
        dz = self.x[2] - atom2.x[2]

        if dx > self.half_L[0]:
            dx = self.L[0] - dx
        if dy > self.half_L[1]:
            dy = self.L[1] - dy
        if dz > self.half_L[2]:
            dz = self.L[2] - dz

        return np.sqrt(dx**2 + dy**2 + dz**2)
    
    def sep_2(self, atom2):
        """ Takes in self and atom2 and finds their separation SQUARED WITHOUT taking into account periodic BCs """

        dx = self.x[0] - atom2.x[0]
        dy = self.x[1] - atom2.x[1]
        dz = self.x[2] - atom2.x[2]

        """
        if dx > self.half_L[0]:
            dx = self.L[0] - dx
        if dy > self.half_L[1]:
            dy = self.L[1] - dy
        if dz > self.half_L[2]:
            dz = self.L[2] - dz
        """

        return dx**2 + dy**2 + dz**2

    def minus(self,B):
        """ Subtract B.x vector from self.x vector """
        
        AminusB = np.array([0.0,0.0,0.0],dtype=np.float64)
        for i in range(3):
            AminusB[i] = self.x[i] - B.x[i]
            
        return AminusB

    def xdot(self,B):
        """ Find dot product of position x of this Atom and Atom B """
        
        AdotB = np.array([0.0,0.0,0.0],dtype=np.float64) # TO DO : find AdotB ??
        
        return AdotB

    # confused as to what this function does :/ - isn't unwrap_flag always False?
    def unwrap(self):
        """ Unwraps the coordinates for periodic box to generate x_unwrap array """
        
        if not self.unwrap_flag:   # first check it has not already been done
            for j in range(3):
                self.x_unwrap[j] = self.x[j] + self.image[j]*self.L[j] # unwrap
            unwrap_flag = True

In [2]:
### functions

def readframe(infile, N):
    """ Read a single frame of N atoms from a dump file 
        Expects coordinates to be in range -L/2 to L/2
        DOES NOT Unwrap corrdinates for periodic box """

    atoms = [Atom() for i in range(N)]
    L = np.array([0.0,0.0,0.0],dtype=np.float64)

    # read in the 9 header lines of the dump file
    # get box size
    for i in range(9):
        line = infile.readline()
        if i==1:  ## second line of frame is timestep
            timestep = np.int32(line)
        if i==5 or i==6 or i==7:   # 6th-8th lines are box size in x,y,z dimensions
            # get the box size
            line = line.split()
            L[i-5]=np.float64(line[1]) - np.float64(line[0]);

    # now read the atoms, putting them at the correct index (index=id-1)
    for i in range(N):
        line = infile.readline()
        line = line.split()
        index = int(line[0])-1  # LAMMPS atom ids start from 1, python index starts from 0
        atoms[index].id = int(line[0])
        atoms[index].type = int(line[1])
        atoms[index].L = L
        for j in range(3):
            atoms[index].x[j] = np.float64(line[j+2])
        for j in range(3):
            atoms[index].image[j] = np.int32(line[j+5])

    return atoms, timestep


def lines_in_file(filename):
    """ Get the number of lines in the file """

    with open(filename) as f:
        for i, l in enumerate(f):
            pass

    return i + 1

In [3]:
### calculation functions

import statistics as s

def dbscan(atoms, threshold, target_type):
    """ Takes in a list of Atom objects, distance threshold + target type. 
        Sets the cluster id for atoms we are not interested in to -2.
        Returns a list of cluster ids where the ith element is the cluster id for ith atom in input list """

    if not atoms: # edge case of empty atom list - return an empty cluster id list
        return []

    cluster_id = 0
    cluster_ids = [-1] * len(atoms)  # Initialize cluster IDs for each atom; -1 means unclassified

    threshold_2 = threshold**2

    def find_neighbors(atom_index):
        """ Takes in index of an atom and find the neighbours of that atom. 
            Atoms are neighbours if within threshold distance of 2.4.
            Returns a list of neighbours of type=4 for atom index inputted """

        return [i for i, other_atom in enumerate(atoms)
                if i != atom_index and atoms[atom_index].type == target_type and other_atom.type == target_type and 
                atoms[atom_index].sep_2(other_atom) < threshold_2]
    
    # loops through atoms list
    for i in range(len(atoms)):

        # checks to see if cluster id of ith atom is -1 - if not atom already processed
        if cluster_ids[i] != -1 or atoms[i].type != target_type:

            # checks to see if atom type is target type - if not sets cluster id of that atom to -2 (not of interest)
            if atoms[i].type != target_type:
                cluster_ids[i] = -2

            continue

        neighbors = find_neighbors(i) # finds neighbours of atom i

        cluster_id += 1
        cluster_ids[i] = cluster_id

        k = 0

        # for every neighbour of atom i, finds the neighbours of that neighbour and adds it to original neighbour list
        while k < len(neighbors):
            neighbor_idx = neighbors[k]
            if cluster_ids[neighbor_idx] == -1:
                cluster_ids[neighbor_idx] = cluster_id

                new_neighbors = find_neighbors(neighbor_idx)
                for new_neighbor in new_neighbors:
                    if cluster_ids[new_neighbor] == -1:
                        neighbors.append(new_neighbor)
            k += 1
            
    no_of_clusters = cluster_id # cluster_id acts like a 'cluster count'

    return no_of_clusters, cluster_ids

In [4]:
def size_of_clusters(cluster_ids):
    """ Takes in a list of cluster ids. Returns a list of the number of proteins in a cluster, ignoring cluster ids of -2.
        Eg: [-2, -2, ..., 1, 4, 2, 4, 1, 3, 4, ...] --> 2 in cluster 1, 1 in cluster 2, 1 in cluster 3, 3 in cluster 4 ... """
    
    size_of_clusters = [0] * max(cluster_ids) # 0 means 0 atoms in cluster i
    
    # loop through list of cluster ids
    for i in cluster_ids:
        
        # if the cluster id is not -2 (-2 are atoms of non-interest)
        if i != -2:
            # increase value of i-1 by 1
            size_of_clusters[i-1] += 1
                    
    return size_of_clusters



def mean_size_of_clusters(size_of_clusters):
    """ Takes in list of cluster sizes (in number of proteins) and returns the mean number of proteins in a cluster """
    
    return s.fmean(size_of_clusters) # fmean runs faster than mean apparently



def size_of_largest_cluster(size_of_clusters):
    """ Takes in list of cluster sizes and returns the number of proteins in largest cluster """
    
    return max(size_of_clusters)


def no_of_clusters_size_1(size_of_clusters):
    """ Takes in a list of cluster sizes and returns the number of clusters with only 1 protein - i.e. size 1 """

    return size_of_clusters.count(1)


def no_proteins_bound_to_poly(atoms, target_types={1, 2, 3}, threshold_2=3.24):
    """ Takes in a list of Atom objects. 
        Returns the no of proteins bound to the polymer of type=1, 2 or 3 + list of no of polymer beads an atom is bound to. """

    def find_polymers_bound_to(j):
        """ Takes in index of a protein (type 4) and finds if it is bound to polymer (type=1, 2 or 3). 
            Bound to polymer if within threshold distance of 1.8. Or if sep_2 < threshold_2 (3.24).
            Returns a list of indices of polymer beads of type=1, 2 or 3 for atom index inputted. 
            Length of list is no of polymer beads that protein is bound to. """

        return [i for i, other_atom in enumerate(atoms)
                #if i != j and other_atom.type in {2} and atoms[j].sep2(other_atom) < threshold_2]
                if i != j and other_atom.type in target_types and atoms[j].sep_2(other_atom) < threshold_2]


    no_proteins_bound = 0 # intialises counter of number of proteins bound to a polymer bead to 0
    no_polymers_bound_to = [0] * len(atoms) # initialises list of number of polymer beads that each protein is bound to, to 0

    # loops through all atoms - could make it loop through just the last 300 as those are the proteins!
    for j, atom in enumerate(atoms):

        # if atom is a protein (type 4)
        if atom.type == 4:

            # find number of polymer beads (of any type - 1, 2 or 3) that protein is bound to
            poly_beads_list = find_polymers_bound_to(j)
            no_polymers_bound_to[j] = len(poly_beads_list) # length of list is the no of polymer beads that protein is bound to

            # if list is not empty
            if poly_beads_list:
                no_proteins_bound += 1

    return no_proteins_bound, no_polymers_bound_to

In [5]:
def fraction_clusters_bound_to_poly(atoms, cluster_ids, cluster_no):
    """ Takes in a list of Atom objects, their cluster ids and no_of_clusters (at each frame). 
        Returns the fraction of clusters bound to the polymer of type=1, 2 or 3 """

    def protein_bound_to_poly(j):
        """ Takes in index of a protein in cluster (type 4) and finds if it is bound to polymer (type=1, 2 or 3). 
            Bound to polymer if within threshold distance of 1.8. Or if sep_2 < threshold_2 (3.24). 
            Stops and returns True if it is bound. False if not bound. """
        for i, other_atom in enumerate(atoms):
            if i != j and other_atom.type in target_types and atoms[j].sep_2(other_atom) < threshold_2:
                return True # stop when you find a protein in cluster that is within threshold distance of any polymer bead
        return False

    bound_clusters = []  # list of cluster ids of bound clusters
    print("bound clusters list: ", bound_clusters)
    target_types={1, 2, 3}
    threshold_2=3.24

    # loop through all cluster ids
    for j, cluster_id in enumerate(cluster_ids):
        
        # if cluster_id is not -2 (ie: it is the cluster id of a protein) and it is not already in bound_clusters
        if cluster_id != -2 and cluster_id not in bound_clusters:

            # check if the protein belonging to that cluster id is bound to a polymer bead. If it is add cluster id to bound_clusters list.
            if protein_bound_to_poly(j):
                bound_clusters.append(cluster_id)

    fraction_bound = len(bound_clusters) / cluster_no # no of bound clusters / total no of clusters
    
    print("---------------------------------------------")
    print("total no of clusters: ",cluster_no)
    print("bound clusters list: ", bound_clusters)
    print("no of bound clusters: ",len(bound_clusters))
    print("frac of bound clusters: ",fraction_bound)
    print("---------------------------------------------")

    return fraction_bound

In [6]:
### main programs --> need to change this cause very sucky!

######################## - User inputs (sys argv command line) but worse cause need to remember the order of inputs :/

"""
dumpfilename = sys.argv[1] # name of dump file to read - dump.sticky_DNA+proteins

Natoms = int(sys.argv[2]) # no of atoms - 220
Npoly = int(sys.argv[3]) # no of polymer atoms - 200

outfile_Rg = sys.argv[4] # name of output file - r_g_sticky_DNA

thresh = float(sys.argv[5]) # cluster threshold - 2.4
"""

######################## - Hardcoded (use for testing)

#name_dumpfile = 'dump.sticky_DNA+proteins' # name of dump file to read - dump.sticky_DNA+proteins

#n_atoms = 220 # no of atoms - 220
#n_poly_atoms = 200 # no of polymer atoms - 200

#name_outfile = 'r_g_sticky_DNA' # name of output file - r_g_sticky_DNA

######################## - User inputs (command line) - best way imo

name_dumpfile = input("Name of dumpfile: ")

n_atoms = int(input("Integer no of atoms (in df): "))
n_poly_atoms = int(input("Integer no of polymer atoms (in df): "))

name_outfile = input("Name of output file: ")

########################

# TO DO: could make this more automatic by giving a default of everything... user can say y/n to default and change if required

threshold = 2.4 # cluster threshold - 2.4
target_type = 4  # target atom type - should change this to type 4 for model simulations... / could add as user input

path_to_dumpfiles = '../../lammps_sims/dumpfiles/' # need to change this when running on local computer
path_to_outfiles = '../outfiles/'

n_lines = lines_in_file(path_to_dumpfiles + name_dumpfile) # no of lines in file
n_frames = int(n_lines / (n_atoms + 9)) # +9 as 9 header lines in each frame

# open the input file
file_in = open(path_to_dumpfiles + name_dumpfile, 'r')

# open the output file and print a header
file_out = open(path_to_outfiles + name_outfile, 'w')
file_out.write("# Timesteps, No of clusters, Fraction of clusters bound\n")

# go through the file frame by frame
for frame in range(n_frames):
    # read the frame, unwrapping periodic coordinates
    atoms, timesteps = readframe(file_in, n_atoms)
    
    # unwarp period boundary coordinates -- needed for clusters? - think so as a cluster can form between boundaries
    for i in range(len(atoms)):
        atoms[i].unwrap()

    # perform calculations
    no_of_clusters, cluster_ids = dbscan(atoms, threshold, target_type)
    print("No of clusters: ", no_of_clusters)
    #cluster_size = size_of_clusters(cluster_ids)
    #mean_cluster_size = mean_size_of_clusters(cluster_size)
    #largest_cluster_size = size_of_largest_cluster(cluster_size)
    
    #no_proteins_bound, no_polymers_bound_to = no_proteins_bound_to_poly(atoms)
    
    frac_clusters_bound = fraction_clusters_bound_to_poly(atoms, cluster_ids, int(no_of_clusters))
    
    # output some results
    #file_out.write("%i %i %i %i\n"%(timesteps, no_of_clusters, mean_cluster_size, largest_cluster_size))
    file_out.write("%i %i %.2f\n"%(timesteps, no_of_clusters, frac_clusters_bound))

# close the files
file_in.close()
file_out.close()

Name of dumpfile: dump_model_4_var_1_run_1.dat
Integer no of atoms (in df): 5300
Integer no of polymer atoms (in df): 5000
Name of output file: test_outfile.dat
No of clusters:  278
bound clusters list:  []
---------------------------------------------
total no of clusters:  278
bound clusters list:  [1, 3, 5, 8, 9, 11, 12, 16, 18, 24, 31, 34, 36, 42, 43, 44, 46, 48, 6, 50, 52, 53, 54, 58, 60, 70, 71, 39, 76, 77, 81, 85, 86, 89, 90, 91, 94, 95, 102, 103, 105, 108, 111, 112, 113, 115, 122, 123, 124, 128, 132, 133, 134, 135, 138, 139, 142, 143, 145, 148, 151, 153, 154, 156, 158, 166, 169, 171, 172, 176, 178, 183, 184, 186, 193, 195, 196, 199, 200, 202, 220, 221, 226, 227, 233, 236, 238, 239, 247, 252, 258, 260, 261, 265, 270, 276]
no of bound clusters:  96
frac of bound clusters:  0.34532374100719426
---------------------------------------------
No of clusters:  224
bound clusters list:  []
---------------------------------------------
total no of clusters:  224
bound clusters list:  [6,

No of clusters:  61
bound clusters list:  []
---------------------------------------------
total no of clusters:  61
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 17, 18, 19, 20, 23, 24, 25, 27, 28, 29, 30, 32, 34, 40, 41, 45, 52, 53, 57, 58, 60, 61]
no of bound clusters:  36
frac of bound clusters:  0.5901639344262295
---------------------------------------------
No of clusters:  59
bound clusters list:  []
---------------------------------------------
total no of clusters:  59
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 17, 19, 20, 21, 22, 24, 25, 26, 27, 29, 30, 31, 33, 36, 46, 47, 49, 50, 54, 55, 56, 57, 59]
no of bound clusters:  37
frac of bound clusters:  0.6271186440677966
---------------------------------------------
No of clusters:  59
bound clusters list:  []
---------------------------------------------
total no of clusters:  59
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 23, 25,

No of clusters:  36
bound clusters list:  []
---------------------------------------------
total no of clusters:  36
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 17, 18, 19, 21, 22, 28, 29, 31, 32, 35]
no of bound clusters:  23
frac of bound clusters:  0.6388888888888888
---------------------------------------------
No of clusters:  33
bound clusters list:  []
---------------------------------------------
total no of clusters:  33
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 14, 15, 17, 18, 19, 20, 25, 26, 27, 29, 32, 33]
no of bound clusters:  22
frac of bound clusters:  0.6666666666666666
---------------------------------------------
No of clusters:  32
bound clusters list:  []
---------------------------------------------
total no of clusters:  32
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 19, 26, 28, 30]
no of bound clusters:  20
frac of bound clusters:  0.625
---------------------------------------------
No of cluste

No of clusters:  20
bound clusters list:  []
---------------------------------------------
total no of clusters:  20
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 18, 19, 20]
no of bound clusters:  17
frac of bound clusters:  0.85
---------------------------------------------
No of clusters:  22
bound clusters list:  []
---------------------------------------------
total no of clusters:  22
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16]
no of bound clusters:  15
frac of bound clusters:  0.6818181818181818
---------------------------------------------
No of clusters:  23
bound clusters list:  []
---------------------------------------------
total no of clusters:  23
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 18, 19, 20, 21]
no of bound clusters:  19
frac of bound clusters:  0.8260869565217391
---------------------------------------------
No of clusters:  23
bound clusters list:  []
------------------------

No of clusters:  21
bound clusters list:  []
---------------------------------------------
total no of clusters:  21
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 17, 18]
no of bound clusters:  12
frac of bound clusters:  0.5714285714285714
---------------------------------------------
No of clusters:  20
bound clusters list:  []
---------------------------------------------
total no of clusters:  20
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 19]
no of bound clusters:  12
frac of bound clusters:  0.6
---------------------------------------------
No of clusters:  19
bound clusters list:  []
---------------------------------------------
total no of clusters:  19
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 15, 16, 18, 19]
no of bound clusters:  15
frac of bound clusters:  0.7894736842105263
---------------------------------------------
No of clusters:  19
bound clusters list:  []
---------------------------------------------
total no of clusters:  19
b

...

No of clusters:  17
bound clusters list:  []
---------------------------------------------
total no of clusters:  17
bound clusters list:  [1, 2, 3, 4, 5, 6, 9, 10, 11, 13, 16]
no of bound clusters:  11
frac of bound clusters:  0.6470588235294118
---------------------------------------------
No of clusters:  16
bound clusters list:  []
---------------------------------------------
total no of clusters:  16
bound clusters list:  [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 15]
no of bound clusters:  12
frac of bound clusters:  0.75
---------------------------------------------
No of clusters:  14
bound clusters list:  []
---------------------------------------------
total no of clusters:  14
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 14]
no of bound clusters:  12
frac of bound clusters:  0.8571428571428571
---------------------------------------------
No of clusters:  14
bound clusters list:  []
---------------------------------------------
total no of clusters:  14
bound cluster

No of clusters:  13
bound clusters list:  []
---------------------------------------------
total no of clusters:  13
bound clusters list:  [1, 2, 3, 4, 5, 6, 8, 9]
no of bound clusters:  8
frac of bound clusters:  0.6153846153846154
---------------------------------------------
No of clusters:  13
bound clusters list:  []
---------------------------------------------
total no of clusters:  13
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]
no of bound clusters:  12
frac of bound clusters:  0.9230769230769231
---------------------------------------------
No of clusters:  15
bound clusters list:  []
---------------------------------------------
total no of clusters:  15
bound clusters list:  [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 13]
no of bound clusters:  11
frac of bound clusters:  0.7333333333333333
---------------------------------------------
No of clusters:  15
bound clusters list:  []
---------------------------------------------
total no of clusters:  15
bound clusters li

No of clusters:  13
bound clusters list:  []
---------------------------------------------
total no of clusters:  13
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13]
no of bound clusters:  12
frac of bound clusters:  0.9230769230769231
---------------------------------------------
No of clusters:  13
bound clusters list:  []
---------------------------------------------
total no of clusters:  13
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13]
no of bound clusters:  12
frac of bound clusters:  0.9230769230769231
---------------------------------------------
No of clusters:  13
bound clusters list:  []
---------------------------------------------
total no of clusters:  13
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 9, 11, 13]
no of bound clusters:  10
frac of bound clusters:  0.7692307692307693
---------------------------------------------
No of clusters:  12
bound clusters list:  []
---------------------------------------------
total no of clusters:  12
bound

No of clusters:  9
bound clusters list:  []
---------------------------------------------
total no of clusters:  9
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
no of bound clusters:  9
frac of bound clusters:  1.0
---------------------------------------------
No of clusters:  9
bound clusters list:  []
---------------------------------------------
total no of clusters:  9
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
no of bound clusters:  9
frac of bound clusters:  1.0
---------------------------------------------
No of clusters:  10
bound clusters list:  []
---------------------------------------------
total no of clusters:  10
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 10]
no of bound clusters:  9
frac of bound clusters:  0.9
---------------------------------------------
No of clusters:  9
bound clusters list:  []
---------------------------------------------
total no of clusters:  9
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9]
no of bound clusters:  9
frac of boun

No of clusters:  13
bound clusters list:  []
---------------------------------------------
total no of clusters:  13
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 9, 12]
no of bound clusters:  9
frac of bound clusters:  0.6923076923076923
---------------------------------------------
No of clusters:  13
bound clusters list:  []
---------------------------------------------
total no of clusters:  13
bound clusters list:  [1, 2, 3, 4, 5, 6, 9, 12, 13]
no of bound clusters:  9
frac of bound clusters:  0.6923076923076923
---------------------------------------------
No of clusters:  12
bound clusters list:  []
---------------------------------------------
total no of clusters:  12
bound clusters list:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
no of bound clusters:  11
frac of bound clusters:  0.9166666666666666
---------------------------------------------
No of clusters:  12
bound clusters list:  []
---------------------------------------------
total no of clusters:  12
bound clusters list:  [1,